## Machine Learning Model Tracking

This is part of a series of tutorials to help you learn how to use Datapane. In this example we use Datapane to produce automated reports of a long-running process, such as an ML pipeline.

# Conducting the Analysis

In [ ]:
# Load the librarys
import pandas as pd  # To work with dataset

# from pandas_profiling import ProfileReport
import numpy as np  # Math library
import json
import datapane as dp
from datetime import datetime
import seaborn as sns  # Graph library that use matplot in background
import matplotlib.pyplot as plt  # to plot some parameters in seaborn
import plotly.offline as py

py.init_notebook_mode(connected=True)  # this code, allow us to work with offline plotly version
import plotly.graph_objs as go  # it's like "plt" of matplot
import plotly.tools as tls  # It's useful to get some tools of plotly
import plotly.figure_factory as ff
from plotly import tools
import plotly.express as px
from sklearn.model_selection import train_test_split, KFold, cross_val_score  # to split the data
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
)  # To evaluate our model
from sklearn.preprocessing import LabelEncoder

# Algorithmns models to be compared
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# from xgboost import XGBClassifier

In [ ]:
# Importing the data
df_credit = pd.read_csv(
    "https://raw.githubusercontent.com/shoumikgoswami/ML-pipeline/main/data/german_credit_data.csv", index_col=0
)

In [ ]:
# Searching for Missings,type of data and also known the shape of data
print(df_credit.info())

# Looking unique values
print(df_credit.nunique())
# Looking the data
print(df_credit.head())

In [ ]:
# enable once ydata-profiling supports 3.11
# profile = ProfileReport(df_credit, title="Data Profile Report")
# json_data = profile.to_json()
# json_data = json.loads(json_data)

In [ ]:
plt.figure(figsize=(14, 12))
corr_plot = sns.heatmap(df_credit.corr(), linewidths=0.1, vmax=1.0, square=True, linecolor="white", annot=True)
plt.show()

In [ ]:
missing_values = df_credit.isnull().sum() * 100 / len(df_credit)
missing_values = missing_values[missing_values > 0]
missing_values.sort_values(inplace=True)
missing_values = missing_values.to_frame()
missing_values.columns = ["% of missing"]
missing_values.index.names = ["Attribute"]
missing_values["Attribute"] = missing_values.index
sns.set(style="whitegrid", color_codes=True)
missing_plot = sns.barplot(x="Attribute", y="% of missing", data=missing_values)
plt.show()

In [ ]:
trace0 = go.Bar(
    x=df_credit[df_credit["Risk"] == "good"]["Risk"].value_counts().index.values,
    y=df_credit[df_credit["Risk"] == "good"]["Risk"].value_counts().values,
    name="Good credit",
)

trace1 = go.Bar(
    x=df_credit[df_credit["Risk"] == "bad"]["Risk"].value_counts().index.values,
    y=df_credit[df_credit["Risk"] == "bad"]["Risk"].value_counts().values,
    name="Bad credit",
)

data = [trace0, trace1]

layout = go.Layout()

layout = go.Layout(yaxis=dict(title="Count"), xaxis=dict(title="Risk Variable"), title="Target variable distribution")

plot1 = go.Figure(data=data, layout=layout)

py.iplot(plot1, filename="grouped-bar")

In [ ]:
df_good = df_credit.loc[df_credit["Risk"] == "good"]["Age"].values.tolist()
df_bad = df_credit.loc[df_credit["Risk"] == "bad"]["Age"].values.tolist()
df_age = df_credit["Age"].values.tolist()

# First plot
trace0 = go.Histogram(x=df_good, histnorm="probability", name="Good Credit")
# Second plot
trace1 = go.Histogram(x=df_bad, histnorm="probability", name="Bad Credit")
# Third plot
trace2 = go.Histogram(x=df_age, histnorm="probability", name="Overall Age")

# Creating the grid
fig = tls.make_subplots(
    rows=2, cols=2, specs=[[{}, {}], [{"colspan": 2}, None]], subplot_titles=("Good", "Bad", "General Distribuition")
)

# setting the figs
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)

fig["layout"].update(showlegend=True, title="Age Distribuition", bargap=0.05)
py.iplot(fig, filename="custom-sized-subplot-with-subplot-titles")
plot2 = fig

In [ ]:
df_good = df_credit[df_credit["Risk"] == "good"]
df_bad = df_credit[df_credit["Risk"] == "bad"]

fig, ax = plt.subplots(nrows=2, figsize=(12, 8))
plt.subplots_adjust(hspace=0.4, top=0.8)

g1 = sns.distplot(df_good["Age"], ax=ax[0], color="g")
g1 = sns.distplot(df_bad["Age"], ax=ax[0], color="r")
g1.set_title("Age Distribuition", fontsize=15)
g1.set_xlabel("Age")
g1.set_xlabel("Frequency")

g2 = sns.countplot(x="Age", data=df_credit, palette="hls", ax=ax[1], hue="Risk")
g2.set_title("Age Counting by Risk", fontsize=15)
g2.set_xlabel("Age")
g2.set_xlabel("Count")
plt.show()
plot3 = g2

In [ ]:
# Let's look the Credit Amount column
interval = (18, 25, 35, 60, 120)

cats = ["Student", "Young", "Adult", "Senior"]
df_credit["Age_cat"] = pd.cut(df_credit.Age, interval, labels=cats)


df_good = df_credit[df_credit["Risk"] == "good"]
df_bad = df_credit[df_credit["Risk"] == "bad"]

In [ ]:
trace0 = go.Box(y=df_good["Credit amount"], x=df_good["Age_cat"], name="Good credit", marker=dict(color="#3D9970"))

trace1 = go.Box(y=df_bad["Credit amount"], x=df_bad["Age_cat"], name="Bad credit", marker=dict(color="#FF4136"))

data = [trace0, trace1]

layout = go.Layout(
    yaxis=dict(title="Credit Amount (US Dollar)", zeroline=False), xaxis=dict(title="Age Categorical"), boxmode="group"
)
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename="box-age-cat")
plot4 = fig

In [ ]:
# First plot
trace0 = go.Bar(
    x=df_credit[df_credit["Risk"] == "good"]["Housing"].value_counts().index.values,
    y=df_credit[df_credit["Risk"] == "good"]["Housing"].value_counts().values,
    name="Good credit",
)

# Second plot
trace1 = go.Bar(
    x=df_credit[df_credit["Risk"] == "bad"]["Housing"].value_counts().index.values,
    y=df_credit[df_credit["Risk"] == "bad"]["Housing"].value_counts().values,
    name="Bad Credit",
)

data = [trace0, trace1]

layout = go.Layout(title="Housing Distribuition")


fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename="Housing-Grouped")
plot5 = fig

In [ ]:
# First plot
trace0 = go.Bar(
    x=df_credit[df_credit["Risk"] == "good"]["Sex"].value_counts().index.values,
    y=df_credit[df_credit["Risk"] == "good"]["Sex"].value_counts().values,
    name="Good credit",
)

# First plot 2
trace1 = go.Bar(
    x=df_credit[df_credit["Risk"] == "bad"]["Sex"].value_counts().index.values,
    y=df_credit[df_credit["Risk"] == "bad"]["Sex"].value_counts().values,
    name="Bad Credit",
)

# Second plot
trace2 = go.Box(
    x=df_credit[df_credit["Risk"] == "good"]["Sex"],
    y=df_credit[df_credit["Risk"] == "good"]["Credit amount"],
    name=trace0.name,
)

# Second plot 2
trace3 = go.Box(
    x=df_credit[df_credit["Risk"] == "bad"]["Sex"],
    y=df_credit[df_credit["Risk"] == "bad"]["Credit amount"],
    name=trace1.name,
)

data = [trace0, trace1, trace2, trace3]


fig = tls.make_subplots(rows=1, cols=2, subplot_titles=("Sex Count", "Credit Amount by Sex"))

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 2)

fig["layout"].update(height=400, width=800, title="Sex Distribuition", boxmode="group")
py.iplot(fig, filename="sex-subplot")

plot6 = fig

In [ ]:
# First plot
trace0 = go.Bar(
    x=df_credit[df_credit["Risk"] == "good"]["Job"].value_counts().index.values,
    y=df_credit[df_credit["Risk"] == "good"]["Job"].value_counts().values,
    name="Good credit Distribuition",
)

# Second plot
trace1 = go.Bar(
    x=df_credit[df_credit["Risk"] == "bad"]["Job"].value_counts().index.values,
    y=df_credit[df_credit["Risk"] == "bad"]["Job"].value_counts().values,
    name="Bad Credit Distribuition",
)

data = [trace0, trace1]

layout = go.Layout(title="Job Distribuition")

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename="grouped-bar")

plot7 = fig

In [ ]:
trace0 = go.Box(x=df_good["Job"], y=df_good["Credit amount"], name="Good credit")

trace1 = go.Box(x=df_bad["Job"], y=df_bad["Credit amount"], name="Bad credit")

data = [trace0, trace1]

layout = go.Layout(yaxis=dict(title="Credit Amount distribuition by Job"), boxmode="group")
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename="box-age-cat")

plot8 = fig

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12), nrows=2)

g1 = sns.boxplot(x="Job", y="Credit amount", data=df_credit, palette="hls", ax=ax[0], hue="Risk")
g1.set_title("Credit Amount by Job", fontsize=15)
g1.set_xlabel("Job Reference", fontsize=12)
g1.set_ylabel("Credit Amount", fontsize=12)

g2 = sns.violinplot(x="Job", y="Age", data=df_credit, ax=ax[1], hue="Risk", split=True, palette="hls")
g2.set_title("Job Type reference x Age", fontsize=15)
g2.set_xlabel("Job Reference", fontsize=12)
g2.set_ylabel("Age", fontsize=12)

plt.subplots_adjust(hspace=0.4, top=0.9)

plt.show()

plot9 = g2

In [ ]:
# Add histogram data
x1 = np.log(df_good["Credit amount"])
x2 = np.log(df_bad["Credit amount"])

# Group data together
hist_data = [x1, x2]

group_labels = ["Good Credit", "Bad Credit"]

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=0.2)

# Plot!
py.iplot(fig, filename="Distplot with Multiple Datasets")

plot10 = fig

In [ ]:
# Ploting the good and bad dataframes in distplot
plt.figure(figsize=(8, 5))

g = sns.distplot(df_good["Credit amount"], color="r")
g = sns.distplot(df_bad["Credit amount"], color="g")
g.set_title("Credit Amount Frequency distribuition", fontsize=15)
plt.show()

In [ ]:
count_good = go.Bar(
    x=df_good["Saving accounts"].value_counts().index.values,
    y=df_good["Saving accounts"].value_counts().values,
    name="Good credit",
)
count_bad = go.Bar(
    x=df_bad["Saving accounts"].value_counts().index.values,
    y=df_bad["Saving accounts"].value_counts().values,
    name="Bad credit",
)


box_1 = go.Box(x=df_good["Saving accounts"], y=df_good["Credit amount"], name="Good credit")
box_2 = go.Box(x=df_bad["Saving accounts"], y=df_bad["Credit amount"], name="Bad credit")

scat_1 = go.Box(x=df_good["Saving accounts"], y=df_good["Age"], name="Good credit")
scat_2 = go.Box(x=df_bad["Saving accounts"], y=df_bad["Age"], name="Bad credit")

data = [scat_1, scat_2, box_1, box_2, count_good, count_bad]

fig = tools.make_subplots(
    rows=2,
    cols=2,
    specs=[[{}, {}], [{"colspan": 2}, None]],
    subplot_titles=("Count Saving Accounts", "Credit Amount by Savings Acc", "Age by Saving accounts"),
)

fig.append_trace(count_good, 1, 1)
fig.append_trace(count_bad, 1, 1)

fig.append_trace(box_2, 1, 2)
fig.append_trace(box_1, 1, 2)

fig.append_trace(scat_1, 2, 1)
fig.append_trace(scat_2, 2, 1)


fig["layout"].update(height=700, width=800, title="Saving Accounts Exploration", boxmode="group")

py.iplot(fig, filename="combined-savings")

plot11 = fig

In [ ]:
print("Description of Distribuition Saving accounts by Risk:  ")
print(pd.crosstab(df_credit["Saving accounts"], df_credit.Risk))

fig, ax = plt.subplots(3, 1, figsize=(12, 12))
g = sns.countplot(x="Saving accounts", data=df_credit, palette="hls", ax=ax[0], hue="Risk")
g.set_title("Saving Accounts Count", fontsize=15)
g.set_xlabel("Saving Accounts type", fontsize=12)
g.set_ylabel("Count", fontsize=12)

g1 = sns.violinplot(x="Saving accounts", y="Job", data=df_credit, palette="hls", hue="Risk", ax=ax[1], split=True)
g1.set_title("Saving Accounts by Job", fontsize=15)
g1.set_xlabel("Savings Accounts type", fontsize=12)
g1.set_ylabel("Job", fontsize=12)

g = sns.boxplot(x="Saving accounts", y="Credit amount", data=df_credit, ax=ax[2], hue="Risk", palette="hls")
g2.set_title("Saving Accounts by Credit Amount", fontsize=15)
g2.set_xlabel("Savings Accounts type", fontsize=12)
g2.set_ylabel("Credit Amount(US)", fontsize=12)

plt.subplots_adjust(hspace=0.4, top=0.9)

plt.show()

plot12 = g2

In [ ]:
print("Values describe: ")
print(pd.crosstab(df_credit.Purpose, df_credit.Risk))

plt.figure(figsize=(14, 12))

plt.subplot(221)
g = sns.countplot(x="Purpose", data=df_credit, palette="hls", hue="Risk")
g.set_xticklabels(g.get_xticklabels(), rotation=45)
g.set_xlabel("", fontsize=12)
g.set_ylabel("Count", fontsize=12)
g.set_title("Purposes Count", fontsize=20)

plt.subplot(222)
g1 = sns.violinplot(x="Purpose", y="Age", data=df_credit, palette="hls", hue="Risk", split=True)
g1.set_xticklabels(g1.get_xticklabels(), rotation=45)
g1.set_xlabel("", fontsize=12)
g1.set_ylabel("Count", fontsize=12)
g1.set_title("Purposes by Age", fontsize=20)

plt.subplot(212)
g2 = sns.boxplot(x="Purpose", y="Credit amount", data=df_credit, palette="hls", hue="Risk")
g2.set_xlabel("Purposes", fontsize=12)
g2.set_ylabel("Credit Amount", fontsize=12)
g2.set_title("Credit Amount distribuition by Purposes", fontsize=20)

plt.subplots_adjust(hspace=0.6, top=0.8)

plt.show()

plot13 = g2

In [ ]:
plt.figure(figsize=(12, 14))

g = plt.subplot(311)
g = sns.countplot(x="Duration", data=df_credit, palette="hls", hue="Risk")
g.set_xlabel("Duration Distribuition", fontsize=12)
g.set_ylabel("Count", fontsize=12)
g.set_title("Duration Count", fontsize=20)

g1 = plt.subplot(312)
g1 = sns.pointplot(x="Duration", y="Credit amount", data=df_credit, hue="Risk", palette="hls")
g1.set_xlabel("Duration", fontsize=12)
g1.set_ylabel("Credit Amount(US)", fontsize=12)
g1.set_title("Credit Amount distribuition by Duration", fontsize=20)

g2 = plt.subplot(313)
g2 = sns.distplot(df_good["Duration"], color="g")
g2 = sns.distplot(df_bad["Duration"], color="r")
g2.set_xlabel("Duration", fontsize=12)
g2.set_ylabel("Frequency", fontsize=12)
g2.set_title("Duration Frequency x good and bad Credit", fontsize=20)

plt.subplots_adjust(wspace=0.4, hspace=0.4, top=0.9)

plt.show()

plot14 = g2

In [ ]:
# First plot
trace0 = go.Bar(
    x=df_credit[df_credit["Risk"] == "good"]["Checking account"].value_counts().index.values,
    y=df_credit[df_credit["Risk"] == "good"]["Checking account"].value_counts().values,
    name="Good credit Distribuition",
)

# Second plot
trace1 = go.Bar(
    x=df_credit[df_credit["Risk"] == "bad"]["Checking account"].value_counts().index.values,
    y=df_credit[df_credit["Risk"] == "bad"]["Checking account"].value_counts().values,
    name="Bad Credit Distribuition",
)

data = [trace0, trace1]

layout = go.Layout(
    title="Checking accounts Distribuition",
    xaxis=dict(title="Checking accounts name"),
    yaxis=dict(title="Count"),
    barmode="group",
)


fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename="Age-ba", validate=False)

plot15 = fig

In [ ]:
df_good = df_credit[df_credit["Risk"] == "good"]
df_bad = df_credit[df_credit["Risk"] == "bad"]

trace0 = go.Box(
    y=df_good["Credit amount"], x=df_good["Checking account"], name="Good credit", marker=dict(color="#3D9970")
)

trace1 = go.Box(
    y=df_bad["Credit amount"], x=df_bad["Checking account"], name="Bad credit", marker=dict(color="#FF4136")
)

data = [trace0, trace1]

layout = go.Layout(yaxis=dict(title="Cheking distribuition"), boxmode="group")
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename="box-age-cat")

plot16 = fig

In [ ]:
print("Total values of the most missing variable: ")
print(df_credit.groupby("Checking account")["Checking account"].count())

plt.figure(figsize=(12, 10))

g = plt.subplot(221)
g = sns.countplot(x="Checking account", data=df_credit, palette="hls", hue="Risk")
g.set_xlabel("Checking Account", fontsize=12)
g.set_ylabel("Count", fontsize=12)
g.set_title("Checking Account Counting by Risk", fontsize=20)

g1 = plt.subplot(222)
g1 = sns.violinplot(x="Checking account", y="Age", data=df_credit, palette="hls", hue="Risk", split=True)
g1.set_xlabel("Checking Account", fontsize=12)
g1.set_ylabel("Age", fontsize=12)
g1.set_title("Age by Checking Account", fontsize=20)

g2 = plt.subplot(212)
g2 = sns.boxplot(x="Checking account", y="Credit amount", data=df_credit, hue="Risk", palette="hls")
g2.set_xlabel("Checking Account", fontsize=12)
g2.set_ylabel("Credit Amount(US)", fontsize=12)
g2.set_title("Credit Amount by Cheking Account", fontsize=20)

plt.subplots_adjust(wspace=0.2, hspace=0.3, top=0.9)

plt.show()
plt.show()

plot17 = g2

In [ ]:
print(pd.crosstab(df_credit.Sex, df_credit.Job))

In [ ]:
plt.figure(figsize=(10, 6))

g = sns.violinplot(x="Housing", y="Job", data=df_credit, hue="Risk", palette="hls", split=True)
g.set_xlabel("Housing", fontsize=12)
g.set_ylabel("Job", fontsize=12)
g.set_title("Housing x Job - Dist", fontsize=20)

plt.show()

plot18 = g

In [ ]:
print(pd.crosstab(df_credit["Checking account"], df_credit.Sex))

In [ ]:
date_int = ["Purpose", "Sex"]
cm = sns.light_palette("green", as_cmap=True)
pd.crosstab(df_credit[date_int[0]], df_credit[date_int[1]]).style.background_gradient(cmap=cm)

In [ ]:
date_int = ["Purpose", "Sex"]
cm = sns.light_palette("green", as_cmap=True)
pd.crosstab(df_credit[date_int[0]], df_credit[date_int[1]]).style.background_gradient(cmap=cm)

In [ ]:
print("Purpose : ", df_credit.Purpose.unique())
print("Sex : ", df_credit.Sex.unique())
print("Housing : ", df_credit.Housing.unique())
print("Saving accounts : ", df_credit["Saving accounts"].unique())
print("Risk : ", df_credit["Risk"].unique())
print("Checking account : ", df_credit["Checking account"].unique())
print("Aget_cat : ", df_credit["Age_cat"].unique())

In [ ]:
def one_hot_encoder(df, nan_as_category=False):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == "object"]
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category, drop_first=True)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [ ]:
df_credit["Saving accounts"] = df_credit["Saving accounts"].fillna("no_inf")
df_credit["Checking account"] = df_credit["Checking account"].fillna("no_inf")

# Purpose to Dummies Variable
df_credit = df_credit.merge(
    pd.get_dummies(df_credit.Purpose, drop_first=True, prefix="Purpose"), left_index=True, right_index=True
)
# Sex feature in dummies
df_credit = df_credit.merge(
    pd.get_dummies(df_credit.Sex, drop_first=True, prefix="Sex"), left_index=True, right_index=True
)
# Housing get dummies
df_credit = df_credit.merge(
    pd.get_dummies(df_credit.Housing, drop_first=True, prefix="Housing"), left_index=True, right_index=True
)
# Housing get Saving Accounts
df_credit = df_credit.merge(
    pd.get_dummies(df_credit["Saving accounts"], drop_first=True, prefix="Savings"), left_index=True, right_index=True
)
# Housing get Risk
df_credit = df_credit.merge(pd.get_dummies(df_credit.Risk, prefix="Risk"), left_index=True, right_index=True)
# Housing get Checking Account
df_credit = df_credit.merge(
    pd.get_dummies(df_credit["Checking account"], drop_first=True, prefix="Check"), left_index=True, right_index=True
)
# Housing get Age categorical
df_credit = df_credit.merge(
    pd.get_dummies(df_credit["Age_cat"], drop_first=True, prefix="Age_cat"), left_index=True, right_index=True
)

In [ ]:
# Excluding the missing columns
del df_credit["Saving accounts"]
del df_credit["Checking account"]
del df_credit["Purpose"]
del df_credit["Sex"]
del df_credit["Housing"]
del df_credit["Age_cat"]
del df_credit["Risk"]
del df_credit["Risk_good"]

In [ ]:
df_credit["Credit amount"] = np.log(df_credit["Credit amount"])

In [ ]:
# Creating the X and y variables
X = df_credit.drop("Risk_bad", 1).values
y = df_credit["Risk_bad"].values
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y)
label_encoded_y = label_encoder.transform(y)
# Spliting X and y into train and test version
X_train, X_test, y_train, y_test = train_test_split(X, label_encoded_y, test_size=0.25, random_state=42)

In [ ]:
# to feed the random state
seed = 7

# prepare models
models = []
models.append(("LR", LogisticRegression()))
models.append(("LDA", LinearDiscriminantAnalysis()))
models.append(("KNN", KNeighborsClassifier()))
models.append(("CART", DecisionTreeClassifier()))
models.append(("NB", GaussianNB()))
models.append(("RF", RandomForestClassifier()))
models.append(("SVM", SVC(gamma="auto")))
# models.append(("XGB", XGBClassifier()))

# evaluate each model in turn
results = []
names = []
scoring = "recall"

for name, model in models:
    kfold = KFold(n_splits=10, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# boxplot algorithm comparison
fig = plt.figure(figsize=(11, 6))
fig.suptitle("Algorithm Comparison")
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

model_plot = fig

In [ ]:
# Criando o classificador logreg
GNB = GaussianNB()

# Fitting with train data
model = GNB.fit(X_train, y_train)

In [ ]:
# Printing the Training Score
print("Training score data: ")
model_score = model.score(X_train, y_train)
print(model_score)

In [ ]:
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))
print("\n")
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

In [ ]:
report = classification_report(y_test, y_pred, output_dict=True)
classification_plot = sns.heatmap(pd.DataFrame(report).iloc[:-1, :].T, annot=True)

In [ ]:
confusion_plot = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True)

In [ ]:
# Predicting proba
y_pred_prob = model.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

fig = px.area(
    x=fpr,
    y=tpr,
    title=f"ROC Curve (AUC={auc(fpr, tpr):.4f})",
    labels=dict(x="False Positive Rate", y="True Positive Rate"),
    width=700,
    height=500,
)
fig.add_shape(type="line", line=dict(dash="dash"), x0=0, x1=1, y0=0, y1=1)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain="domain")
fig.show()

roc_plot = fig

# Datapane

### Blocks

In [ ]:
# Text blocks
update_commentary = dp.Text(
    """
## Update commentary
This is a placeholder commentary section to display updates on the training and changes in the model. The section can provide a high level idea on what has changed and its impact on business.
"""
)

context = dp.Text(
    """
## Objective
This project has been created as a demo to showcase how Datapane can be used to track, infer and share machine learning pipeline updates. The report has been created as an HTML dashboard which can be easily updated and shared across.
"""
)

content = dp.Text(
    """
## Content
The original dataset contains 1000 entries with 20 categorial/symbolic attributes prepared by Prof. Hofmann. In this dataset, each entry represents a person who takes a credit by a bank. Each person is classified as good or bad credit risks according to the set of attributes.

- Age (numeric)
- Sex (text: male, female)
- Job (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)
- Housing (text: own, rent, or free)
- Saving accounts (text - little, moderate, quite rich, rich)
- Checking account (numeric, in DM - Deutsch Mark)
- Credit amount (numeric, in DM)
- Duration (numeric, in month)
- Purpose (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)
"""
)

acknowledgement = dp.Text(
    """
## Acknowledgements
Source = https://www.kaggle.com/kabure/predicting-credit-risk-model-pipeline/data
"""
)

In [ ]:
# Big numbers
report_update = dp.BigNumber(heading="Report last updated", value=datetime.now().strftime("%Y-%m-%d %H:%M"))

accuracy_block = dp.BigNumber(heading="Model accuracy", value="65%", change="2%", is_upward_change=True)

fscore_block = dp.BigNumber(heading="F1-score", value="66%", change="1%", is_upward_change=False)

trainingtime_block = dp.BigNumber(heading="Model training time", value="130ms", change="3%", is_upward_change=False)

model_training_score = dp.BigNumber(
    heading="Model training score (%)", value=round(model_score * 100, 2), change="5%", is_upward_change=True
)

In [ ]:
# Groups - waiting on ydata-profiling support
# overview_block = dp.Group(
#     dp.BigNumber(heading="Number of variables", value=json_data["table"]["n_var"]),
#     dp.BigNumber(heading="Number of observations", value=json_data["table"]["n"]),
#     dp.BigNumber(heading="Missing cells", value=json_data["table"]["n_cells_missing"]),
#     dp.BigNumber(heading="Missing cells (%)", value=round(json_data["table"]["p_cells_missing"] * 100, 2)),
#     dp.BigNumber(heading="Duplicate rows", value=json_data["table"]["n_duplicates"]),
#     dp.BigNumber(heading="Duplicate rows (%)", value=round(json_data["table"]["p_duplicates"] * 100, 2)),
#     dp.BigNumber(heading="Total size in memory (KB)", value=json_data["table"]["memory_size"] / 1000),
#     dp.BigNumber(heading="Numeric variables", value=json_data["table"]["types"]["Numeric"]),
#     dp.BigNumber(heading="Categorical variables", value=json_data["table"]["types"]["Categorical"]),
#     columns=3,
#     label="Overview",
# )

### Pages

In [ ]:
page1 = dp.Page(
    title="Summary",
    blocks=[
        dp.Group(
            dp.Text("![](https://raw.githubusercontent.com/shoumikgoswami/ML-pipeline/main/assets/dp_credit.png)"),
            dp.Group(report_update, update_commentary, columns=1),
            columns=2,
        ),
        dp.Group(accuracy_block, fscore_block, trainingtime_block, columns=3),
    ],
)

page2 = dp.Page(title="Project details", blocks=[context, content, acknowledgement])

page3 = dp.Page(
    title="Data profile",
    blocks=[
        dp.Text("""## Dataset details"""),
        dp.Text(
            """This page can be used to display an overview of the dataset along with important elements of the data, features, attributes with null values and so on. """
        ),
        dp.Select(
            blocks=[
                # overview_block,
                dp.Plot(corr_plot, label="Correlation"),
                dp.Plot(missing_plot, label="Missing values"),
                dp.DataTable(df_credit, label="Dataset"),
            ]
        ),
    ],
)

page4 = dp.Page(
    title="Exploration",
    blocks=[
        dp.Text("""## Data exploration"""),
        dp.Text(
            """This page can be used to display plots and visualization elements used during the data exploration phase. """
        ),
        dp.Plot(plot1),
        dp.Plot(plot2),
        dp.Plot(plot3),
        dp.Plot(plot4),
        dp.Plot(plot5),
        dp.Plot(plot6),
        dp.Plot(plot7),
        dp.Plot(plot8),
        dp.Plot(plot9),
        dp.Plot(plot10),
        dp.Plot(plot11),
        dp.Plot(plot12),
        dp.Plot(plot13),
        dp.Plot(plot14),
        dp.Plot(plot15),
        dp.Plot(plot16),
        dp.Plot(plot17),
        dp.Plot(plot18),
    ],
)


page5 = dp.Page(
    title="Model tracking",
    blocks=[
        dp.Text("""## Model tracking and logging"""),
        dp.Text("""This page can be used to track models, changes in the scores and performance."""),
        dp.Text("""### Comparing performance of different models"""),
        dp.Plot(model_plot),
        model_training_score,
        dp.Text("""### ROC curve based on the chosen model"""),
        dp.Plot(roc_plot),
        dp.Group(
            dp.Group(dp.Text("""### Classification report matrix"""), dp.Plot(classification_plot), columns=1),
            dp.Group(dp.Text("""### Confusion matrix"""), dp.Plot(confusion_plot), columns=1),
            columns=2,
        ),
    ],
)

### Report

In [ ]:
report = dp.Blocks(page1, page2, page3, page4, page5)
dp.save_report(report, path="report.html", open=True)

# uncomment to upload to DPCloud
# dp.upload_report(report, name="Machine learning tracking and monitoring system", open=True)